In [1]:
#!pip install --upgrade google-cloud-bigquery[pandas]

In [2]:
from google.cloud import bigquery

In [3]:
client=bigquery.Client()

C:\ProgramData\Anaconda3\lib\site-packages\google\auth\_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
%load_ext google.cloud.bigquery

In [5]:
%reload_ext google.cloud.bigquery

In [6]:
#######################################
print('Setting everything up...')
#######################################

import warnings
warnings.filterwarnings('ignore')
import pandas_gbq 
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.lines import Line2D

import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl

import matplotlib.pyplot as plt
%matplotlib inline



import os
import sys
from datetime import datetime
from datetime import date
from datetime import time
from datetime import timedelta
import time

DATASET = 'aou-res-curation-prod.prod_drc_dataset'
 
plt.style.use('ggplot')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

from IPython.display import HTML as html_print

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)


print('done.')


Setting everything up...
done.


In [7]:
dic={'src_hpo_id':["seec_miami","seec_morehouse","seec_emory","uamc_banner","pitt","nyc_cu","ipmc_uic","trans_am_spectrum","tach_hfhs","nec_bmc","cpmc_uci","nec_phs","nyc_cornell","ipmc_nu","nyc_hh","ipmc_uchicago","aouw_mcri","syhc","cpmc_ceders","seec_ufl","saou_uab","trans_am_baylor","cpmc_ucsd","ecchc","chci","aouw_uwh","cpmc_usc","hrhc","ipmc_northshore","chs","cpmc_ucsf","jhchc","aouw_mcw","cpmc_ucd","ipmc_rush"],
    'HPO':["SouthEast Enrollment Center Miami","SouthEast Enrollment Center Morehouse","SouthEast Enrollment Center Emory","Banner Health","University of Pittsburgh","Columbia University Medical Center","University of Illinois Chicago","Spectrum Health","Henry Ford Health System","Boston Medical Center","UC Irvine","Partners HealthCare","Weill Cornell Medical Center","Northwestern Memorial Hospital","Harlem Hospital","University of Chicago","Marshfield Clinic","San Ysidro Health Center","Cedars-Sinai","University of Florida","University of Alabama at Birmingham","Baylor","UC San Diego","Eau Claire Cooperative Health Center","Community Health Center, Inc.","UW Health (University of Wisconsin Madison)","University of Southern California","HRHCare","NorthShore University Health System","Cherokee Health Systems","UC San Francisco","Jackson-Hinds CHC","Medical College of Wisconsin","UC Davis","Rush University"]}


site_df=pd.DataFrame(data=dic)
site_df

,src_hpo_id,HPO
0,seec_miami,SouthEast Enrollment Center Miami
1,seec_morehouse,SouthEast Enrollment Center Morehouse
2,seec_emory,SouthEast Enrollment Center Emory
3,uamc_banner,Banner Health
4,pitt,University of Pittsburgh
5,nyc_cu,Columbia University Medical Center
6,ipmc_uic,University of Illinois Chicago
7,trans_am_spectrum,Spectrum Health
8,tach_hfhs,Henry Ford Health System
9,nec_bmc,Boston Medical Center


# Foreign key references (i.e. visit_occurrence_id in the condition table) should be valid.

## Person

In [8]:
print(("There is no _mapping table for person table so I could not separete results by sites "))

There is no _mapping table for person table so I could not separete results by sites 


### gender_concept_id

In [9]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        gender_concept_id,
        concept_name,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_person` AS p
    INNER JOIN
        `{}.concept` AS c
        ON
            p.gender_concept_id=c.concept_id
    GROUP BY
        1,2
    '''.format(DATASET, DATASET,DATASET,DATASET, DATASET,DATASET),
                    dialect='standard')
print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
6 records received.


In [10]:
foreign_key_df

,gender_concept_id,concept_name,cnt
0,8551,UNKNOWN,11
1,8507,MALE,41031
2,8532,FEMALE,68562
3,0,No matching concept,46
4,4215271,Gender unspecified,10
5,4214687,Gender unknown,1


In [11]:
foreign_key_df.loc[foreign_key_df["gender_concept_id"]==0,["cnt"]]

,cnt
3,46


In [12]:
success_rate=100-round(100*(foreign_key_df.loc[foreign_key_df["gender_concept_id"]==0,["cnt"]])/sum(foreign_key_df.iloc[:,2]),1)
print("success rate for gender_concept_id is: ", success_rate.iloc[0,0]) 

success rate for gender_concept_id is:  100.0


In [13]:
foreign_key_df

,gender_concept_id,concept_name,cnt
0,8551,UNKNOWN,11
1,8507,MALE,41031
2,8532,FEMALE,68562
3,0,No matching concept,46
4,4215271,Gender unspecified,10
5,4214687,Gender unknown,1


### race_concept_id

In [14]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        race_concept_id,
        concept_name,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_person` AS p
    INNER JOIN
        `{}.concept` AS c
        ON
            p.race_concept_id=c.concept_id
    GROUP BY
        1,2
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
31 records received.


In [15]:
foreign_key_df

,race_concept_id,concept_name,cnt
0,8522,Other Race,2473
1,8552,Unknown,1796
2,44814649,Other,7
3,44814653,Unknown,4221
4,4184966,American Indian or Alaska native,9
5,8516,Black or African American,18221
6,38003615,Middle Eastern or North African,9
7,38003598,Black,1774
8,8527,White,63201
9,38003584,Japanese,14


In [16]:
success_rate=100-round(100*(foreign_key_df.loc[foreign_key_df["race_concept_id"]==0,["cnt"]])/sum(foreign_key_df.iloc[:,2]),1)
print("success rate for race_concept_id is: ", success_rate.iloc[0,0]) 

success rate for race_concept_id is:  89.9


### ethnicity_concept_id

In [17]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        ethnicity_concept_id,
        concept_name,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_person` AS p
    INNER JOIN
        `{}.concept` AS c
        ON
            p.ethnicity_concept_id=c.concept_id
    GROUP BY
        1,2
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
4 records received.


In [18]:
foreign_key_df

,ethnicity_concept_id,concept_name,cnt
0,38003563,Hispanic or Latino,17101
1,38003564,Not Hispanic or Latino,68194
2,8552,Unknown,161
3,0,No matching concept,24205


In [19]:
success_rate=100-round(100*(foreign_key_df.loc[foreign_key_df["ethnicity_concept_id"]==0,["cnt"]])/sum(foreign_key_df.iloc[:,2]),1)
print("success rate for ethnicity_concept_id is: ", round(success_rate.iloc[0,0],1)) 

success rate for ethnicity_concept_id is:  77.9


### location_id

In [20]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
    location_id,
    COUNT(*) AS total_cnt
    FROM
       `{}.unioned_ehr_person` AS p
    GROUP BY
        1
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
43719 records received.


In [21]:
foreign_key_df.head()

,location_id,total_cnt
0,3.500000e+16,1
1,3.500000e+16,1
2,3.500000e+16,1
3,3.500000e+16,1
4,3.500000e+16,1


In [22]:

print("location_id is NULL ") 

location_id is NULL 


### provider_id

In [23]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        provider_id,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_person` AS p
    GROUP BY
        1
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
7384 records received.


In [24]:
foreign_key_df.head()

,provider_id,cnt
0,15090.0,4
1,NaN,44222
2,42414.0,1
3,13562.0,4
4,12902.0,17


In [25]:

print("provider_id is NULL ") 

provider_id is NULL 


### care_site_id

In [26]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        care_site_id,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_person` AS p
    GROUP BY
        1
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
1169 records received.


In [27]:
foreign_key_df.head()

,care_site_id,cnt
0,3.500000e+16,26
1,3.500000e+16,294
2,3.500000e+16,4
3,3.500000e+16,5
4,3.500000e+16,36


In [28]:

print("care_site_id is NULL ") 

care_site_id is NULL 


### gender_source_concept_id

In [29]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        gender_source_concept_id,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_person` AS p
    GROUP BY
        1
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
15 records received.


In [30]:
foreign_key_df

,gender_source_concept_id,cnt
0,0.0,10410
1,8532.0,6841
2,8507.0,4716
3,NaN,44865
4,1.0,2822
5,2.0,1741
6,4214687.0,1
7,362.0,14874
8,363.0,10489
9,44814666.0,277


In [31]:

print("gender_source_concept_id is NULL ") 

gender_source_concept_id is NULL 


### race_source_concept_id

In [32]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        race_source_concept_id,
        concept_name,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_person` AS p
    INNER JOIN
        `{}.concept` AS c
        ON
            p.race_source_concept_id=c.concept_id
    GROUP BY
        1,2
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
46 records received.


In [33]:
foreign_key_df

,race_source_concept_id,concept_name,cnt
0,11,Procedure type,4079
1,9,Visit type,3
2,6,Death type,479
3,8522,Other Race,809
4,8552,Unknown,792
5,8516,Black or African American,4479
6,38003615,Middle Eastern or North African,4
7,38003598,Black,248
8,8527,White,12099
9,201,Quantified Clinical Drug Box,2


In [34]:

print("race_source_concept_id is NULL ") 

race_source_concept_id is NULL 


### ethnicity_source_concept_id

In [35]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        ethnicity_source_concept_id,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_person` AS p
    GROUP BY
        1
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
52 records received.


In [36]:
foreign_key_df

,ethnicity_source_concept_id,cnt
0,0.000000e+00,23447
1,3.800356e+07,6237
2,NaN,29301
3,8.552000e+03,288
4,3.000000e+00,2558
5,1.000000e+00,3837
6,3.800356e+07,976
7,2.000000e+00,2549
8,4.000000e+00,3549
9,1.100000e+01,2


In [37]:

print("ethnicity_source_concept_id is NULL ") 

ethnicity_source_concept_id is NULL 


## VISIT_OCCURANCE TABLE

### person_id

In [38]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        COUNT(*) AS total,
        sum(case when (vo.person_id is null or vo.person_id=0) then 1 else 0 end) as missing
    FROM
       `{}.unioned_ehr_visit_occurrence` AS vo
    INNER JOIN
        `{}.unioned_ehr_observation` AS o
        ON
            vo.person_id=o.person_id
    WHERE 
        o.observation_source_concept_id=1586099 and o.value_as_concept_id=45877994
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
1 records received.


In [39]:
foreign_key_df

,total,missing
0,0,None


### visit_concept_id

In [40]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        concept_name,
        visit_concept_id,
        COUNT(*)
    FROM
       `{}.unioned_ehr_visit_occurrence` AS vo
    INNER JOIN
        `{}.concept` AS c
        ON
            vo.visit_concept_id=c.concept_id
    GROUP BY
        1,2
    ORDER BY
        2
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
21 records received.


In [41]:
foreign_key_df.head()

,concept_name,visit_concept_id,f0_
0,No matching concept,0,1730719
1,Emergency Room and Inpatient Visit,262,15471
2,Off Campus-Outpatient Hospital,5084,6210
3,Inpatient Hospital,8717,22296
4,Outpatient Hospital,8756,116797


#### visit_concept_id BY SITE

In [42]:
######################################
print('Getting the data from the database...')
######################################

visit_occurrence_visit_concept_id_df = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(vo.person_id) as total_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                `{}.concept` AS c
                ON
                    vo.visit_concept_id=c.concept_id
            LEFT OUTER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                `{}.concept` AS c
                ON
                    vo.visit_concept_id=c.concept_id
            LEFT OUTER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            WHERE
                (vo.visit_concept_id is null or vo.visit_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
visit_occurrence_visit_concept_id_df.shape

print(visit_occurrence_visit_concept_id_df.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [43]:
visit_occurrence_visit_concept_id_df

,src_hpo_id,missing_counts,total_counts,succes_rate
0,ecchc,NaN,12077,NaN
1,ipmc_northshore,NaN,94179,NaN
2,tach_hfhs,NaN,628057,NaN
3,seec_morehouse,NaN,59855,NaN
4,nyc_hh,NaN,19334,NaN
5,hrhc,NaN,40251,NaN
6,chci,NaN,89741,NaN
7,syhc,NaN,78498,NaN
8,saou_uab,NaN,206058,NaN
9,ipmc_rush,NaN,56208,NaN


In [44]:
visit_occurrence_visit_concept_id_df=visit_occurrence_visit_concept_id_df.rename(columns={"succes_rate":"visit_occurrence_visit_concept_id"})
visit_occurrence_visit_concept_id_df=visit_occurrence_visit_concept_id_df[["src_hpo_id","visit_occurrence_visit_concept_id"]]
visit_occurrence_visit_concept_id_df=visit_occurrence_visit_concept_id_df.fillna(100)
visit_occurrence_visit_concept_id_df

,src_hpo_id,visit_occurrence_visit_concept_id
0,ecchc,100.0
1,ipmc_northshore,100.0
2,tach_hfhs,100.0
3,seec_morehouse,100.0
4,nyc_hh,100.0
5,hrhc,100.0
6,chci,100.0
7,syhc,100.0
8,saou_uab,100.0
9,ipmc_rush,100.0


### visit_type_concept_id

In [45]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        concept_name,
        visit_type_concept_id,
        COUNT(*)
    FROM
       `{}.unioned_ehr_visit_occurrence` AS vo
    INNER JOIN
        `{}.concept` AS c
        ON
            vo.visit_type_concept_id=c.concept_id
    GROUP BY
        1,2
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
6 records received.


In [46]:
foreign_key_df

,concept_name,visit_type_concept_id,f0_
0,Non-hospital institution Visit,42898160,1309
1,No matching concept,0,218789
2,Inpatient Visit,9201,29739
3,Outpatient Visit,9202,723910
4,Emergency Room Visit,9203,20767
5,Visit derived from EHR record,44818518,11963399


In [47]:
success_rate=100-round(100*(foreign_key_df.loc[foreign_key_df["visit_type_concept_id"]==0,["f0_"]])/sum(foreign_key_df.iloc[:,2]),1)
print("success rate for visit_concept_id is: ", success_rate.iloc[0,0]) 

success rate for visit_concept_id is:  98.3


#### visit_type_concept_id BY SITE

In [48]:
######################################
print('Getting the data from the database...')
######################################

visit_occurrence_visit_type_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(vo.person_id) as total_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                `{}.concept` AS c
                ON
                    vo.visit_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                `{}.concept` AS c
                ON
                    vo.visit_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            WHERE
                (vo.visit_type_concept_id is null or vo.visit_type_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
visit_occurrence_visit_type_concept_id.shape

print(visit_occurrence_visit_type_concept_id.shape[0], 'records received.')

Getting the data from the database...
31 records received.


In [49]:
visit_occurrence_visit_type_concept_id

,src_hpo_id,missing_counts,total_counts,succes_rate
0,uamc_banner,NaN,390178,NaN
1,hrhc,NaN,40251,NaN
2,seec_emory,NaN,110137,NaN
3,cpmc_ucsd,NaN,343573,NaN
4,jhchc,NaN,24192,NaN
5,cpmc_uci,NaN,328582,NaN
6,nec_bmc,NaN,547560,NaN
7,chs,NaN,63840,NaN
8,tach_hfhs,NaN,628057,NaN
9,cpmc_ucd,NaN,291202,NaN


In [50]:
visit_occurrence_visit_type_concept_id=visit_occurrence_visit_type_concept_id.rename(columns={"succes_rate":"visit_occurrence_visit_type_concept_id"})
visit_occurrence_visit_type_concept_id=visit_occurrence_visit_type_concept_id[["src_hpo_id","visit_occurrence_visit_type_concept_id"]]
visit_occurrence_visit_type_concept_id=visit_occurrence_visit_type_concept_id.fillna(100)
visit_occurrence_visit_type_concept_id

,src_hpo_id,visit_occurrence_visit_type_concept_id
0,uamc_banner,100.0
1,hrhc,100.0
2,seec_emory,100.0
3,cpmc_ucsd,100.0
4,jhchc,100.0
5,cpmc_uci,100.0
6,nec_bmc,100.0
7,chs,100.0
8,tach_hfhs,100.0
9,cpmc_ucd,100.0


### provider_id

In [51]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        vo.provider_id,
        COUNT(*) AS cnt
    FROM
       `{}.unioned_ehr_visit_occurrence` AS vo
    GROUP BY
        1
    ORDER BY
        2    
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
66573 records received.


In [52]:
foreign_key_df.tail(10)

,provider_id,cnt
66563,3967237.0,23449
66564,2882158.0,25677
66565,1934.0,27611
66566,-1.0,27645
66567,4.0,35739
66568,51143.0,43119
66569,3965876.0,47971
66570,3.0,54120
66571,0.0,1546655
66572,NaN,6892827


In [53]:
100-round(100*(foreign_key_df.loc[foreign_key_df["provider_id"].isnull(),["cnt"]].iloc[0,0]
+foreign_key_df.loc[(foreign_key_df["provider_id"]==0),["cnt"]].iloc[0,0])/sum(foreign_key_df.iloc[:,1]),1)

35.400000000000006

In [54]:
total_missing=foreign_key_df.loc[foreign_key_df["provider_id"].isnull(),["cnt"]].iloc[0,0]+foreign_key_df.loc[(foreign_key_df["provider_id"]==0),["cnt"]].iloc[0,0]
total_missing

8439482

#### provider_id by sites

In [55]:
######################################
print('Getting the data from the database...')
######################################

visit_occurrence_provider_id_df = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as total_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
visit_occurrence_provider_id_df.shape

print(visit_occurrence_provider_id_df.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [56]:
visit_occurrence_provider_id_df

,src_hpo_id,missing_counts,total_counts,succes_rate
0,ecchc,12077,12077,0.0
1,uamc_banner,390178,390178,0.0
2,cpmc_ucsf,95602,95602,0.0
3,tach_hfhs,628057,628057,0.0
4,nec_phs,3895972,3895972,0.0
5,cpmc_uci,328582,328582,0.0
6,nec_bmc,547560,547560,0.0
7,chs,63840,63840,0.0
8,chci,89741,89741,0.0
9,seec_morehouse,136615,136615,0.0


In [57]:
visit_occurrence_provider_id_df=visit_occurrence_provider_id_df.rename(columns={"succes_rate":"visit_occurrence_provider_id"})
visit_occurrence_provider_id_df=visit_occurrence_provider_id_df[["src_hpo_id","visit_occurrence_provider_id"]]
visit_occurrence_provider_id_df=visit_occurrence_provider_id_df.fillna(100)
visit_occurrence_provider_id_df

,src_hpo_id,visit_occurrence_provider_id
0,ecchc,0.0
1,uamc_banner,0.0
2,cpmc_ucsf,0.0
3,tach_hfhs,0.0
4,nec_phs,0.0
5,cpmc_uci,0.0
6,nec_bmc,0.0
7,chs,0.0
8,chci,0.0
9,seec_morehouse,0.0


#### care_site_id BY SITE

In [58]:
######################################
print('Getting the data from the database...')
######################################

visit_occurrence_care_site_id_df = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(vo.person_id) as total_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            WHERE
                (vo.care_site_id is null or vo.care_site_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(visit_occurrence_care_site_id_df.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [59]:
visit_occurrence_care_site_id_df=visit_occurrence_care_site_id_df.rename(columns={"succes_rate":"visit_occurrence_care_site_id"})
visit_occurrence_care_site_id_df=visit_occurrence_care_site_id_df[["src_hpo_id","visit_occurrence_care_site_id"]]
visit_occurrence_care_site_id_df=visit_occurrence_care_site_id_df.fillna(100)
visit_occurrence_care_site_id_df

,src_hpo_id,visit_occurrence_care_site_id
0,hrhc,100.0
1,cpmc_usc,100.0
2,saou_uab,100.0
3,cpmc_ucd,100.0
4,trans_am_spectrum,100.0
5,chci,100.0
6,aouw_uwh,100.0
7,aouw_mcri,100.0
8,nec_phs,100.0
9,chs,100.0


#### visit_source_concept_id BY SITE

In [60]:
######################################
print('Getting the data from the database...')
######################################

visit_occurrence_visit_source_concept_id_df = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(vo.person_id) as total_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            WHERE
                (vo.visit_source_concept_id is null or vo.visit_source_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
visit_occurrence_visit_source_concept_id_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
66573 records received.


In [61]:
visit_occurrence_visit_source_concept_id_df=visit_occurrence_visit_source_concept_id_df.rename(columns={"succes_rate":"visit_occurrence_visit_source_concept_id"})
visit_occurrence_visit_source_concept_id_df=visit_occurrence_visit_source_concept_id_df[["src_hpo_id","visit_occurrence_visit_source_concept_id"]]
visit_occurrence_visit_source_concept_id_df=visit_occurrence_visit_source_concept_id_df.fillna(100)
visit_occurrence_visit_source_concept_id_df

,src_hpo_id,visit_occurrence_visit_source_concept_id
0,hrhc,100.0
1,cpmc_uci,100.0
2,seec_morehouse,100.0
3,seec_ufl,100.0
4,pitt,100.0
5,nec_bmc,0.0
6,cpmc_ucsf,0.0
7,ipmc_rush,0.0
8,ipmc_northshore,0.0
9,jhchc,0.0


#### admitting_source_concept_id BY SITE

In [62]:
######################################
print('Getting the data from the database...')
######################################

visit_occurrence_admitting_source_concept_id_df = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(vo.person_id) as total_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            WHERE
                (vo.admitting_source_concept_id is null or vo.admitting_source_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
visit_occurrence_admitting_source_concept_id_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
66573 records received.


In [63]:
visit_occurrence_admitting_source_concept_id_df=visit_occurrence_admitting_source_concept_id_df.rename(columns={"succes_rate":"visit_occurrence_admitting_source_concept_id"})
visit_occurrence_admitting_source_concept_id_df=visit_occurrence_admitting_source_concept_id_df[["src_hpo_id","visit_occurrence_admitting_source_concept_id"]]
visit_occurrence_admitting_source_concept_id_df=visit_occurrence_admitting_source_concept_id_df.fillna(100)
visit_occurrence_admitting_source_concept_id_df

,src_hpo_id,visit_occurrence_admitting_source_concept_id
0,cpmc_usc,100.0
1,nyc_hh,100.0
2,ecchc,0.0
3,nec_phs,0.0
4,chs,0.0
5,cpmc_uci,0.0
6,chci,0.0
7,hrhc,0.0
8,saou_uab,0.0
9,seec_miami,0.0


#### discharge_to_concept_id BY SITE

In [64]:
######################################
print('Getting the data from the database...')
######################################

visit_occurrence_discharge_to_concept_id_df = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(vo.person_id) as total_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                `{}.concept` AS c
                ON
                    vo.discharge_to_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                `{}.concept` AS c
                ON
                    vo.discharge_to_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            WHERE
                (vo.discharge_to_concept_id is null or vo.discharge_to_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
visit_occurrence_discharge_to_concept_id_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
66573 records received.


In [65]:
visit_occurrence_discharge_to_concept_id_df=visit_occurrence_discharge_to_concept_id_df.rename(columns={"succes_rate":"visit_occurrence_discharge_to_concept_id"})
visit_occurrence_discharge_to_concept_id_df=visit_occurrence_discharge_to_concept_id_df[["src_hpo_id","visit_occurrence_discharge_to_concept_id"]]
visit_occurrence_discharge_to_concept_id_df=visit_occurrence_discharge_to_concept_id_df.fillna(100)
visit_occurrence_discharge_to_concept_id_df

,src_hpo_id,visit_occurrence_discharge_to_concept_id
0,nyc_cu,100.0
1,chs,0.0
2,tach_hfhs,0.0
3,seec_morehouse,0.0
4,uamc_banner,0.0
5,jhchc,0.0
6,seec_emory,0.0
7,saou_uab,0.0
8,seec_miami,0.0
9,aouw_mcw,0.0


#### preceding_visit_occurrence_id BY SITE

In [66]:
######################################
print('Getting the data from the database...')
######################################

visit_occurrence_preceding_visit_occurrence_id_df = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(vo.person_id) as total_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                vo.visit_occurrence_id=mvo.visit_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mvo.src_hpo_id,
                COUNT(vo.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_visit_occurrence` AS vo
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_visit_occurrence`) AS mvo
                ON
                    vo.visit_occurrence_id=mvo.visit_occurrence_id
            WHERE
                (vo.preceding_visit_occurrence_id is null or vo.preceding_visit_occurrence_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
visit_occurrence_preceding_visit_occurrence_id_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
66573 records received.


In [67]:
visit_occurrence_preceding_visit_occurrence_id_df=visit_occurrence_preceding_visit_occurrence_id_df.rename(columns={"succes_rate":"visit_occurrence_preceding_visit_occurrence_id"})
visit_occurrence_preceding_visit_occurrence_id_df=visit_occurrence_preceding_visit_occurrence_id_df[["src_hpo_id","visit_occurrence_preceding_visit_occurrence_id"]]
visit_occurrence_preceding_visit_occurrence_id_df=visit_occurrence_preceding_visit_occurrence_id_df.fillna(100)
visit_occurrence_preceding_visit_occurrence_id_df

,src_hpo_id,visit_occurrence_preceding_visit_occurrence_id
0,cpmc_usc,100.0
1,ecchc,0.0
2,cpmc_ucsd,0.0
3,jhchc,0.0
4,seec_emory,0.0
5,cpmc_ucsf,0.0
6,tach_hfhs,0.0
7,seec_morehouse,0.0
8,uamc_banner,0.0
9,cpmc_uci,0.0


## Condition Occurrence Table

#### condition_concept_id BY SITE

In [68]:
######################################
print('Getting the data from the database...')
######################################

condition_occurrence_condition_concept_id_df = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(co.person_id) as total_counts
            FROM
               `{}.unioned_ehr_condition_occurrence` AS co
            INNER JOIN
                `{}.concept` AS c
                ON
                    co.condition_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                    `{}._mapping_condition_occurrence`)  AS mco
                ON
                    co.condition_occurrence_id=mco.condition_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mco.src_hpo_id,
                COUNT(co.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_condition_occurrence` AS co
            INNER JOIN
                `{}.concept` AS c
                ON
                    co.condition_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                    `{}._mapping_condition_occurrence`)  AS mco
                ON
                    co.condition_occurrence_id=mco.condition_occurrence_id
            WHERE
                (co.condition_concept_id is null or co.condition_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
condition_occurrence_condition_concept_id_df.shape

print(condition_occurrence_condition_concept_id_df.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [69]:
condition_occurrence_condition_concept_id_df=condition_occurrence_condition_concept_id_df.rename(columns={"succes_rate":"condition_occurrence_condition_concept_id"})
condition_occurrence_condition_concept_id_df=condition_occurrence_condition_concept_id_df[["src_hpo_id","condition_occurrence_condition_concept_id"]]
condition_occurrence_condition_concept_id_df=condition_occurrence_condition_concept_id_df.fillna(100)
condition_occurrence_condition_concept_id_df

,src_hpo_id,condition_occurrence_condition_concept_id
0,saou_uab,100.0
1,trans_am_spectrum,100.0
2,cpmc_ceders,100.0
3,cpmc_ucsd,100.0
4,seec_emory,100.0
5,jhchc,100.0
6,nec_bmc,100.0
7,syhc,100.0
8,aouw_mcw,100.0
9,ipmc_rush,100.0


#### condition_type_concept_id BY SITE

In [70]:
######################################
print('Getting the data from the database...')
######################################

condition_occurrence_condition_type_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(co.person_id) as total_counts
            FROM
               `{}.unioned_ehr_condition_occurrence` AS co
            INNER JOIN
                `{}.concept` AS c
                ON
                    co.condition_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                    `{}._mapping_condition_occurrence`)  AS mco
                ON
                    co.condition_occurrence_id=mco.condition_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mco.src_hpo_id,
                COUNT(co.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_condition_occurrence` AS co
            INNER JOIN
                `{}.concept` AS c
                ON
                    co.condition_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                    `{}._mapping_condition_occurrence`)  AS mco
                ON
                    co.condition_occurrence_id=mco.condition_occurrence_id
            WHERE
                (co.condition_type_concept_id is null or co.condition_type_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
condition_occurrence_condition_type_concept_id.shape

print(condition_occurrence_condition_type_concept_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [71]:
condition_occurrence_condition_type_concept_id=condition_occurrence_condition_type_concept_id.rename(columns={"succes_rate":"condition_occurrence_condition_type_concept_id"})
condition_occurrence_condition_type_concept_id=condition_occurrence_condition_type_concept_id[["src_hpo_id","condition_occurrence_condition_type_concept_id"]]
condition_occurrence_condition_type_concept_id=condition_occurrence_condition_type_concept_id.fillna(100)
condition_occurrence_condition_type_concept_id

,src_hpo_id,condition_occurrence_condition_type_concept_id
0,ipmc_northshore,100.0
1,trans_am_spectrum,100.0
2,hrhc,100.0
3,nyc_hh,100.0
4,jhchc,100.0
5,cpmc_ucsf,100.0
6,cpmc_uci,100.0
7,chci,100.0
8,nyc_cu,100.0
9,syhc,100.0


### Provider_id


In [72]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        co.provider_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_condition_occurrence` AS co
    GROUP BY
        1
    ORDER BY
        2
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
56628 records received.


In [73]:
foreign_key_df.tail()

,provider_id,cnt
56623,998.0,143981
56624,5180.0,179330
56625,14026.0,460137
56626,0.0,1793752
56627,NaN,11286469


### visit_occurrence_id

In [74]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        co.visit_occurrence_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_condition_occurrence` AS co
    GROUP BY
        1
    ORDER BY
        2
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
5641625 records received.


In [75]:
foreign_key_df.tail()

,visit_occurrence_id,cnt
5641620,3.400000e+16,2274
5641621,3.400000e+16,2313
5641622,3.400000e+16,2506
5641623,3.400000e+16,3234
5641624,NaN,2939583


### condition_source_concept_id

In [76]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        condition_source_concept_id,
        COUNT(*) as cnt
    FROM
       `{}.unioned_ehr_condition_occurrence` AS co
    GROUP BY
        1
    ORDER BY
        2
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
67066 records received.


In [77]:
foreign_key_df.tail()

,condition_source_concept_id,cnt
67061,4274025.0,290974
67062,4042140.0,321139
67063,441840.0,493779
67064,NaN,740371
67065,0.0,1046584


In [78]:
success_rate=100-round(100*(foreign_key_df.loc[foreign_key_df["condition_source_concept_id"].isnull(),["cnt"]].iloc[0,0]
+foreign_key_df.loc[(foreign_key_df["condition_source_concept_id"]==0),["cnt"]].iloc[0,0])/sum(foreign_key_df.iloc[:,1]),1)
print("success rate for condition_source_concept_id is: ", round(success_rate,1)) 

success rate for condition_source_concept_id is:  93.7


#### condition_source_concept_id BY SITE

In [79]:
######################################
print('Getting the data from the database...')
######################################

condition_occurrence_condition_source_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(co.person_id) as total_counts
            FROM
               `{}.unioned_ehr_condition_occurrence` AS co
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                    `{}._mapping_condition_occurrence`)  AS mco
                ON
                    co.condition_occurrence_id=mco.condition_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mco.src_hpo_id,
                COUNT(co.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_condition_occurrence` AS co
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                    `{}._mapping_condition_occurrence`)  AS mco
                ON
                    co.condition_occurrence_id=mco.condition_occurrence_id
            WHERE
                (co.condition_source_concept_id is null or co.condition_source_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
condition_occurrence_condition_source_concept_id.shape

print(condition_occurrence_condition_source_concept_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [80]:
condition_occurrence_condition_source_concept_id=condition_occurrence_condition_source_concept_id.rename(columns={"succes_rate":"condition_occurrence_condition_source_concept_id"})
condition_occurrence_condition_source_concept_id=condition_occurrence_condition_source_concept_id[["src_hpo_id","condition_occurrence_condition_source_concept_id"]]
condition_occurrence_condition_source_concept_id=condition_occurrence_condition_source_concept_id.fillna(100)
condition_occurrence_condition_source_concept_id

,src_hpo_id,condition_occurrence_condition_source_concept_id
0,cpmc_uci,100.0
1,nec_phs,100.0
2,chs,100.0
3,tach_hfhs,100.0
4,chci,100.0
5,ecchc,100.0
6,nyc_cornell,100.0
7,trans_am_spectrum,100.0
8,syhc,100.0
9,seec_morehouse,100.0


#### condition_status_concept_id BY SITE

In [81]:
######################################
print('Getting the data from the database...')
######################################

condition_occurrence_condition_status_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(co.person_id) as total_counts
            FROM
               `{}.unioned_ehr_condition_occurrence` AS co
            INNER JOIN
                `{}.concept` AS c
                ON
                    co.condition_status_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                    `{}._mapping_condition_occurrence`)  AS mco
                ON
                    co.condition_occurrence_id=mco.condition_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mco.src_hpo_id,
                COUNT(co.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_condition_occurrence` AS co
            INNER JOIN
                `{}.concept` AS c
                ON
                    co.condition_status_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                    `{}._mapping_condition_occurrence`)  AS mco
                ON
                    co.condition_occurrence_id=mco.condition_occurrence_id
            WHERE
                (co.condition_status_concept_id is null or co.condition_status_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
condition_occurrence_condition_status_concept_id.shape

print(condition_occurrence_condition_status_concept_id.shape[0], 'records received.')

Getting the data from the database...
19 records received.


In [82]:
condition_occurrence_condition_status_concept_id=condition_occurrence_condition_status_concept_id.rename(columns={"succes_rate":"condition_occurrence_condition_status_concept_id"})
condition_occurrence_condition_status_concept_id=condition_occurrence_condition_status_concept_id[["src_hpo_id","condition_occurrence_condition_status_concept_id"]]
condition_occurrence_condition_status_concept_id=condition_occurrence_condition_status_concept_id.fillna(100)
condition_occurrence_condition_status_concept_id

,src_hpo_id,condition_occurrence_condition_status_concept_id
0,ipmc_rush,100.0
1,chs,100.0
2,seec_morehouse,100.0
3,nec_phs,100.0
4,nyc_cu,100.0
5,seec_emory,100.0
6,nyc_hh,100.0
7,trans_am_spectrum,100.0
8,nyc_cornell,100.0
9,tach_hfhs,0.0


## Drug Exposure Table

### person_id

In [83]:
######################################
print('Getting the data from the database...')
######################################

foreign_key_df = pd.io.gbq.read_gbq('''
    SELECT
        COUNT(*) AS total,
        sum(case when (de.person_id is null or de.person_id=0) then 1 else 0 end) as missing
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    INNER JOIN
        `{}.unioned_ehr_observation` AS o
        ON
            de.person_id=o.person_id
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
foreign_key_df.shape

print(foreign_key_df.shape[0], 'records received.')

Getting the data from the database...
1 records received.


In [84]:
foreign_key_df

,total,missing
0,4738855973,0


In [85]:

print("success rate for person_id is: ",round(100-100*(foreign_key_df.iloc[0,1]/foreign_key_df.iloc[0,0]),1))

success rate for person_id is:  100.0


#### drug_concept_id BY SITE

In [86]:
######################################
print('Getting the data from the database...')
######################################

drug_exposure_drug_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(de.person_id) as total_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                `{}.concept` AS c
                ON
                    de.drug_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mde.src_hpo_id,
                COUNT(de.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                `{}.concept` AS c
                ON
                    de.drug_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            WHERE
                (de.drug_concept_id is null or de.drug_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
drug_exposure_drug_concept_id.shape

print(drug_exposure_drug_concept_id.shape[0], 'records received.')

Getting the data from the database...
33 records received.


In [87]:
drug_exposure_drug_concept_id=drug_exposure_drug_concept_id.rename(columns={"succes_rate":"drug_exposure_drug_concept_id"})
drug_exposure_drug_concept_id=drug_exposure_drug_concept_id[["src_hpo_id","drug_exposure_drug_concept_id"]]
drug_exposure_drug_concept_id=drug_exposure_drug_concept_id.fillna(100)
drug_exposure_drug_concept_id

,src_hpo_id,drug_exposure_drug_concept_id
0,ecchc,100.0
1,nyc_cornell,100.0
2,aouw_mcw,100.0
3,hrhc,100.0
4,nyc_hh,100.0
5,nec_bmc,100.0
6,cpmc_ceders,100.0
7,pitt,100.0
8,jhchc,100.0
9,trans_am_spectrum,100.0


#### drug_type_concept_id BY SITE

In [88]:
######################################
print('Getting the data from the database...')
######################################

drug_exposure_drug_type_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(de.person_id) as total_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                `{}.concept` AS c
                ON
                    de.drug_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mde.src_hpo_id,
                COUNT(de.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                `{}.concept` AS c
                ON
                    de.drug_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            WHERE
                (de.drug_type_concept_id is null or de.drug_type_concept_id=0) 
            GROUP BY
                1
        )
    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
drug_exposure_drug_type_concept_id.shape

print(drug_exposure_drug_type_concept_id.shape[0], 'records received.')

Getting the data from the database...
33 records received.


In [89]:
drug_exposure_drug_type_concept_id=drug_exposure_drug_type_concept_id.rename(columns={"succes_rate":"condition_occurrence_drug_type_concept_id"})
drug_exposure_drug_type_concept_id=drug_exposure_drug_type_concept_id[["src_hpo_id","condition_occurrence_drug_type_concept_id"]]
drug_exposure_drug_type_concept_id=drug_exposure_drug_type_concept_id.fillna(100)
drug_exposure_drug_type_concept_id

,src_hpo_id,condition_occurrence_drug_type_concept_id
0,chs,100.0
1,chci,100.0
2,tach_hfhs,100.0
3,cpmc_uci,100.0
4,nec_bmc,100.0
5,ecchc,100.0
6,nyc_cornell,100.0
7,aouw_mcri,100.0
8,nec_phs,100.0
9,ipmc_uic,100.0


#### route_concept_id BY SITE

In [90]:
######################################
print('Getting the data from the database...')
######################################

drug_exposure_route_concept_id= pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(de.person_id) as total_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                `{}.concept` AS c
                ON
                    de.route_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mde.src_hpo_id,
                COUNT(de.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                `{}.concept` AS c
                ON
                    de.route_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            WHERE
                (de.route_concept_id is null or de.route_concept_id=0) 
            GROUP BY
                1
        )
    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
drug_exposure_route_concept_id.shape

print(drug_exposure_route_concept_id.shape[0], 'records received.')

Getting the data from the database...
27 records received.


In [91]:
drug_exposure_route_concept_id=drug_exposure_route_concept_id.rename(columns={"succes_rate":"drug_exposure_route_concept_id"})
drug_exposure_route_concept_id=drug_exposure_route_concept_id[["src_hpo_id","drug_exposure_route_concept_id"]]
drug_exposure_route_concept_id=drug_exposure_route_concept_id.fillna(100)
drug_exposure_route_concept_id

,src_hpo_id,drug_exposure_route_concept_id
0,nyc_cu,100.0
1,ipmc_nu,100.0
2,jhchc,100.0
3,cpmc_usc,0.0
4,cpmc_ucd,0.0
5,ipmc_rush,0.0
6,syhc,0.0
7,seec_emory,0.0
8,saou_uab,0.0
9,pitt,17.5


#### provider_id BY SITE

In [92]:
######################################
print('Getting the data from the database...')
######################################

drug_exposure_provider_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(de.person_id) as total_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mde.src_hpo_id,
                COUNT(de.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                (SELECT
                    DISTINCT * 
                    FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            WHERE
                (de.provider_id is null or de.provider_id=0) 
            GROUP BY
                1
        )
    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
drug_exposure_provider_id.shape

print(drug_exposure_provider_id.shape[0], 'records received.')

Getting the data from the database...
33 records received.


In [93]:
drug_exposure_provider_id=drug_exposure_provider_id.rename(columns={"succes_rate":"drug_exposure_provider_id"})
drug_exposure_provider_id=drug_exposure_provider_id[["src_hpo_id","drug_exposure_provider_id"]]
drug_exposure_provider_id=drug_exposure_provider_id.fillna(100)
drug_exposure_provider_id

,src_hpo_id,drug_exposure_provider_id
0,chs,100.0
1,ipmc_rush,100.0
2,hrhc,100.0
3,nec_phs,0.0
4,cpmc_uci,0.0
5,chci,0.0
6,nyc_cu,0.0
7,nyc_hh,0.0
8,nyc_cornell,0.0
9,cpmc_ceders,0.0


#### visit_occurrence_id BY SITE

In [94]:
######################################
print('Getting the data from the database...')
######################################

drug_exposure_visit_occurrence_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(de.person_id) as total_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                (SELECT
                    DISTINCT * 
                    FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mde.src_hpo_id,
                COUNT(de.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                (SELECT
                    DISTINCT * 
                    FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            WHERE
                (de.visit_occurrence_id is null or de.visit_occurrence_id=0) 
            GROUP BY
                1
        )
    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
drug_exposure_visit_occurrence_id.shape

print(drug_exposure_visit_occurrence_id.shape[0], 'records received.')

Getting the data from the database...
33 records received.


In [95]:
drug_exposure_visit_occurrence_id=drug_exposure_visit_occurrence_id.rename(columns={"succes_rate":"drug_exposure_visit_occurrence_id"})
drug_exposure_visit_occurrence_id=drug_exposure_visit_occurrence_id[["src_hpo_id","drug_exposure_visit_occurrence_id"]]
drug_exposure_visit_occurrence_id=drug_exposure_visit_occurrence_id.fillna(100)
drug_exposure_visit_occurrence_id

,src_hpo_id,drug_exposure_visit_occurrence_id
0,ipmc_rush,100.0
1,cpmc_ucsf,100.0
2,uamc_banner,100.0
3,ipmc_uic,100.0
4,trans_am_spectrum,100.0
5,cpmc_usc,100.0
6,chci,100.0
7,jhchc,100.0
8,cpmc_ucsd,100.0
9,syhc,100.0


#### drug_source_concept_id BY SITE

In [96]:
######################################
print('Getting the data from the database...')
######################################

drug_exposure_drug_source_concept_id= pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT
                src_hpo_id,
                COUNT(de.person_id) as total_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mde.src_hpo_id,
                COUNT(de.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_drug_exposure` AS de
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS mde
                ON
                    de.drug_exposure_id=mde.drug_exposure_id
            WHERE
                (de.drug_source_concept_id is null or de.drug_source_concept_id=0) 
            GROUP BY
                1
        )
    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
drug_exposure_drug_source_concept_id.shape

print(drug_exposure_drug_source_concept_id.shape[0], 'records received.')

Getting the data from the database...
33 records received.


In [97]:
drug_exposure_drug_source_concept_id=drug_exposure_drug_source_concept_id.rename(columns={"succes_rate":"drug_exposure_drug_source_concept_id"})
drug_exposure_drug_source_concept_id=drug_exposure_drug_source_concept_id[["src_hpo_id","drug_exposure_drug_source_concept_id"]]
drug_exposure_drug_source_concept_id=drug_exposure_drug_source_concept_id.fillna(100)
drug_exposure_drug_source_concept_id

,src_hpo_id,drug_exposure_drug_source_concept_id
0,ecchc,100.0
1,chci,100.0
2,seec_ufl,100.0
3,pitt,100.0
4,nyc_hh,100.0
5,hrhc,100.0
6,cpmc_ceders,100.0
7,trans_am_spectrum,100.0
8,cpmc_uci,100.0
9,nyc_cornell,100.0


## Measurement table

#### measurement_concept_id BY SITE

In [98]:
######################################
print('Getting the data from the database...')
######################################

measurement_measurement_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(me.person_id) as total_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.measurement_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mm.src_hpo_id,
                COUNT(me.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.measurement_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            WHERE
                (me.measurement_concept_id is null or me.measurement_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
measurement_measurement_concept_id.shape

print(measurement_measurement_concept_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [99]:
measurement_measurement_concept_id=measurement_measurement_concept_id.rename(columns={"succes_rate":"measurement_measurement_concept_id"})
measurement_measurement_concept_id=measurement_measurement_concept_id[["src_hpo_id","measurement_measurement_concept_id"]]
measurement_measurement_concept_id=measurement_measurement_concept_id.fillna(100)
measurement_measurement_concept_id

,src_hpo_id,measurement_measurement_concept_id
0,aouw_mcw,100.0
1,cpmc_ceders,100.0
2,nec_bmc,100.0
3,pitt,100.0
4,chci,100.0
5,cpmc_ucsd,100.0
6,cpmc_ucsf,100.0
7,saou_uab,100.0
8,ipmc_rush,100.0
9,syhc,100.0


#### measurement_type_concept_id BY SITE

In [100]:
######################################
print('Getting the data from the database...')
######################################

measurement_measurement_type_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(me.person_id) as total_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.measurement_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mm.src_hpo_id,
                COUNT(me.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.measurement_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            WHERE
                (me.measurement_type_concept_id is null or me.measurement_type_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
measurement_measurement_type_concept_id.shape

print(measurement_measurement_type_concept_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [101]:
measurement_measurement_type_concept_id=measurement_measurement_type_concept_id.rename(columns={"succes_rate":"measurement_measurement_type_concept_id"})
measurement_measurement_type_concept_id=measurement_measurement_type_concept_id[["src_hpo_id","measurement_measurement_type_concept_id"]]
measurement_measurement_type_concept_id=measurement_measurement_type_concept_id.fillna(100)
measurement_measurement_type_concept_id

,src_hpo_id,measurement_measurement_type_concept_id
0,nyc_hh,100.0
1,hrhc,100.0
2,cpmc_uci,100.0
3,cpmc_ceders,100.0
4,cpmc_ucsd,100.0
5,jhchc,100.0
6,cpmc_ucsf,100.0
7,chs,100.0
8,nyc_cornell,100.0
9,saou_uab,100.0


#### operator_concept_id BY SITE

In [102]:
######################################
print('Getting the data from the database...')
######################################

measurement_operator_concept_id= pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(me.person_id) as total_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.operator_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mm.src_hpo_id,
                COUNT(me.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.operator_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            WHERE
                (me.operator_concept_id is null or me.operator_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
measurement_operator_concept_id.shape

print(measurement_operator_concept_id.shape[0], 'records received.')

Getting the data from the database...
28 records received.


In [103]:
measurement_operator_concept_id=measurement_operator_concept_id.rename(columns={"succes_rate":"measurement_operator_concept_id"})
measurement_operator_concept_id=measurement_operator_concept_id[["src_hpo_id","measurement_operator_concept_id"]]
measurement_operator_concept_id=measurement_operator_concept_id.fillna(100)
measurement_operator_concept_id

,src_hpo_id,measurement_operator_concept_id
0,seec_emory,100.0
1,nyc_cu,100.0
2,ipmc_rush,100.0
3,seec_miami,100.0
4,aouw_uwh,100.0
5,trans_am_spectrum,100.0
6,ipmc_uic,100.0
7,seec_ufl,100.0
8,cpmc_ucsd,0.0
9,cpmc_uci,0.0


#### value_as_concept_id BY SITE

In [104]:
######################################
print('Getting the data from the database...')
######################################

measurement_value_as_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(me.person_id) as total_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.value_as_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mm.src_hpo_id,
                COUNT(me.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.value_as_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            WHERE
                (me.value_as_concept_id is null or me.value_as_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
measurement_value_as_concept_id.shape

print(measurement_value_as_concept_id.shape[0], 'records received.')

Getting the data from the database...
29 records received.


In [105]:
measurement_value_as_concept_id=measurement_value_as_concept_id.rename(columns={"succes_rate":"measurement_value_as_concept_id"})
measurement_value_as_concept_id=measurement_value_as_concept_id[["src_hpo_id","measurement_value_as_concept_id"]]
measurement_value_as_concept_id=measurement_value_as_concept_id.fillna(100)
measurement_value_as_concept_id

,src_hpo_id,measurement_value_as_concept_id
0,cpmc_ucsf,100.0
1,cpmc_ucd,100.0
2,nyc_cu,100.0
3,ecchc,0.0
4,nec_bmc,0.0
5,seec_ufl,0.0
6,ipmc_rush,0.0
7,cpmc_ceders,0.0
8,cpmc_usc,0.0
9,nyc_cornell,0.0


#### unit_concept_id BY SITE

In [106]:
######################################
print('Getting the data from the database...')
######################################

measurement_unit_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(me.person_id) as total_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.unit_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mm.src_hpo_id,
                COUNT(me.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                `{}.concept` AS c
                ON
                    me.unit_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            WHERE
                (me.unit_concept_id is null or me.unit_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
measurement_unit_concept_id.shape

print(measurement_unit_concept_id.shape[0], 'records received.')

Getting the data from the database...
29 records received.


In [107]:
measurement_unit_concept_id=measurement_unit_concept_id.rename(columns={"succes_rate":"measurement_unit_concept_id"})
measurement_unit_concept_id=measurement_unit_concept_id[["src_hpo_id","measurement_unit_concept_id"]]
measurement_unit_concept_id=measurement_unit_concept_id.fillna(100)
measurement_unit_concept_id

,src_hpo_id,measurement_unit_concept_id
0,aouw_mcw,100.0
1,ipmc_nu,100.0
2,cpmc_ucsf,100.0
3,aouw_uwh,100.0
4,chci,100.0
5,cpmc_ucd,100.0
6,cpmc_uci,0.0
7,cpmc_usc,0.0
8,ipmc_rush,0.0
9,nec_bmc,0.0


#### provider_id BY SITE

In [108]:
######################################
print('Getting the data from the database...')
######################################

measurement_provider_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(me.person_id) as total_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mm.src_hpo_id,
                COUNT(me.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            WHERE
                (me.provider_id is null or me.provider_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        3
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
measurement_provider_id.shape

print(measurement_provider_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [109]:
measurement_provider_id=measurement_provider_id.rename(columns={"succes_rate":"measurement_provider_id"})
measurement_provider_id=measurement_provider_id[["src_hpo_id","measurement_provider_id"]]
measurement_provider_id=measurement_provider_id.fillna(100)
measurement_provider_id

,src_hpo_id,measurement_provider_id
0,nyc_hh,0.0
1,seec_morehouse,98.9
2,hrhc,100.0
3,ecchc,0.0
4,jhchc,96.5
5,seec_emory,14.5
6,ipmc_rush,0.0
7,chs,100.0
8,chci,0.0
9,cpmc_ceders,0.0


#### visit_occurrence_id BY SITE

In [110]:
######################################
print('Getting the data from the database...')
######################################

measurement_visit_occurrence_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(me.person_id) as total_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mm.src_hpo_id,
                COUNT(me.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            WHERE
                (me.visit_occurrence_id is null or me.visit_occurrence_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        3
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
measurement_visit_occurrence_id.shape

print(measurement_visit_occurrence_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [111]:
measurement_visit_occurrence_id=measurement_visit_occurrence_id.rename(columns={"succes_rate":"measurement_visit_occurrence_id"})
measurement_visit_occurrence_id=measurement_visit_occurrence_id[["src_hpo_id","measurement_visit_occurrence_id"]]
measurement_visit_occurrence_id=measurement_visit_occurrence_id.fillna(100)
measurement_visit_occurrence_id

,src_hpo_id,measurement_visit_occurrence_id
0,nyc_hh,95.2
1,seec_morehouse,100.0
2,hrhc,100.0
3,ecchc,0.2
4,jhchc,100.0
5,seec_emory,100.0
6,ipmc_rush,100.0
7,chs,100.0
8,chci,100.0
9,cpmc_ceders,0.0


#### measurement_source_concept_id BY SITE

In [112]:
######################################
print('Getting the data from the database...')
######################################

measurement_measurement_source_concept_id= pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(me.person_id) as total_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                mm.src_hpo_id,
                COUNT(me.person_id) as missing_counts
            FROM
               `{}.unioned_ehr_measurement` AS me
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS mm
                ON
                    me.measurement_id=mm.measurement_id
            WHERE
                (me.measurement_source_concept_id is null or me.measurement_source_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        3
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
measurement_measurement_source_concept_id.shape

print(measurement_measurement_source_concept_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [113]:
measurement_measurement_source_concept_id=measurement_measurement_source_concept_id.rename(columns={"succes_rate":"measurement_measurement_source_concept_id"})
measurement_measurement_source_concept_id=measurement_measurement_source_concept_id[["src_hpo_id","measurement_measurement_source_concept_id"]]
measurement_measurement_source_concept_id=measurement_measurement_source_concept_id.fillna(100)
measurement_measurement_source_concept_id

,src_hpo_id,measurement_measurement_source_concept_id
0,nyc_hh,100.0
1,seec_morehouse,96.3
2,hrhc,100.0
3,ecchc,88.5
4,jhchc,100.0
5,seec_emory,99.9
6,ipmc_rush,100.0
7,chs,87.7
8,chci,100.0
9,cpmc_ceders,100.0


## Procedure Occurrence

#### procedure_concept_id BY SITE

In [114]:
######################################
print('Getting the data from the database...')
######################################

procedure_occurrence_procedure_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.procedure_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.procedure_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            WHERE
                (t1.procedure_concept_id is null or t1.procedure_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
procedure_occurrence_procedure_concept_id.shape

print(procedure_occurrence_procedure_concept_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [115]:
procedure_occurrence_procedure_concept_id=procedure_occurrence_procedure_concept_id.rename(columns={"succes_rate":"procedure_occurrence_procedure_concept_id"})
procedure_occurrence_procedure_concept_id=procedure_occurrence_procedure_concept_id[["src_hpo_id","procedure_occurrence_procedure_concept_id"]]
procedure_occurrence_procedure_concept_id=procedure_occurrence_procedure_concept_id.fillna(100)
procedure_occurrence_procedure_concept_id

,src_hpo_id,procedure_occurrence_procedure_concept_id
0,cpmc_uci,100.0
1,nec_bmc,100.0
2,aouw_mcw,100.0
3,seec_ufl,100.0
4,ipmc_rush,100.0
5,ipmc_northshore,100.0
6,pitt,100.0
7,trans_am_spectrum,100.0
8,chs,100.0
9,hrhc,100.0


#### procedure_type_concept_id BY SITE

In [116]:
######################################
print('Getting the data from the database...')
######################################

procedure_occurrence_procedure_type_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.procedure_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.procedure_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            WHERE
                (t1.procedure_type_concept_id is null or t1.procedure_type_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
procedure_occurrence_procedure_type_concept_id.shape

print(procedure_occurrence_procedure_type_concept_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [117]:
procedure_occurrence_procedure_type_concept_id=procedure_occurrence_procedure_type_concept_id.rename(columns={"succes_rate":"procedure_occurrence_procedure_type_concept_id"})
procedure_occurrence_procedure_type_concept_id=procedure_occurrence_procedure_type_concept_id[["src_hpo_id","procedure_occurrence_procedure_type_concept_id"]]
procedure_occurrence_procedure_type_concept_id=procedure_occurrence_procedure_type_concept_id.fillna(100)
procedure_occurrence_procedure_type_concept_id

,src_hpo_id,procedure_occurrence_procedure_type_concept_id
0,ipmc_northshore,100.0
1,ecchc,100.0
2,nyc_cu,100.0
3,chs,100.0
4,jhchc,100.0
5,aouw_mcri,100.0
6,syhc,100.0
7,nyc_cornell,100.0
8,nyc_hh,100.0
9,hrhc,100.0


#### modifier_concept_id BY SITE

In [118]:
######################################
print('Getting the data from the database...')
######################################

procedure_occurrence_modifier_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.modifier_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.modifier_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            WHERE
                (t1.modifier_concept_id is null or t1.modifier_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
procedure_occurrence_modifier_concept_id.shape

print(procedure_occurrence_modifier_concept_id.shape[0], 'records received.')

Getting the data from the database...
26 records received.


In [119]:
procedure_occurrence_modifier_concept_id=procedure_occurrence_modifier_concept_id.rename(columns={"succes_rate":"procedure_occurrence_modifier_concept_id"})
procedure_occurrence_modifier_concept_id=procedure_occurrence_modifier_concept_id[["src_hpo_id","procedure_occurrence_modifier_concept_id"]]
procedure_occurrence_modifier_concept_id=procedure_occurrence_modifier_concept_id.fillna(100)
procedure_occurrence_modifier_concept_id

,src_hpo_id,procedure_occurrence_modifier_concept_id
0,nyc_cu,100.0
1,cpmc_ucsf,100.0
2,cpmc_ceders,100.0
3,cpmc_uci,0.0
4,ipmc_uic,0.0
5,nec_bmc,0.0
6,chci,0.0
7,ipmc_northshore,0.0
8,uamc_banner,0.0
9,ecchc,0.0


#### provider_id BY SITE

In [120]:
######################################
print('Getting the data from the database...')
######################################

procedure_occurrence_provider_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            WHERE
                (t1.provider_id is null or t1.provider_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
procedure_occurrence_provider_id.shape

print(procedure_occurrence_provider_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [121]:
procedure_occurrence_provider_id=procedure_occurrence_provider_id.rename(columns={"succes_rate":"procedure_occurrence_provider_id"})
procedure_occurrence_provider_id=procedure_occurrence_provider_id[["src_hpo_id","procedure_occurrence_provider_id"]]
procedure_occurrence_provider_id=procedure_occurrence_provider_id.fillna(100)
procedure_occurrence_provider_id

,src_hpo_id,procedure_occurrence_provider_id
0,aouw_mcw,100.0
1,hrhc,100.0
2,nec_phs,0.0
3,chci,0.0
4,ecchc,0.0
5,ipmc_rush,0.0
6,nyc_cornell,0.0
7,saou_uab,0.0
8,cpmc_uci,0.0
9,cpmc_ucsd,0.0


#### visit_occurrence_id BY SITE

In [122]:
######################################
print('Getting the data from the database...')
######################################

procedure_occurrence_visit_occurrence_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            WHERE
                (t1.visit_occurrence_id is null or t1.visit_occurrence_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
procedure_occurrence_visit_occurrence_id.shape

print(procedure_occurrence_visit_occurrence_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [123]:
procedure_occurrence_visit_occurrence_id=procedure_occurrence_visit_occurrence_id.rename(columns={"succes_rate":"procedure_occurrence_visit_occurrence_id"})
procedure_occurrence_visit_occurrence_id=procedure_occurrence_visit_occurrence_id[["src_hpo_id","procedure_occurrence_visit_occurrence_id"]]
procedure_occurrence_visit_occurrence_id=procedure_occurrence_visit_occurrence_id.fillna(100)
procedure_occurrence_visit_occurrence_id

,src_hpo_id,procedure_occurrence_visit_occurrence_id
0,ipmc_northshore,100.0
1,chci,100.0
2,nec_phs,100.0
3,chs,100.0
4,cpmc_ucsf,100.0
5,nec_bmc,100.0
6,cpmc_ucsd,100.0
7,seec_emory,100.0
8,jhchc,100.0
9,nyc_hh,100.0


#### procedure_source_concept_id BY SITE

In [124]:
######################################
print('Getting the data from the database...')
######################################

procedure_occurrence_procedure_source_concept_id= pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_procedure_occurrence` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_procedure_occurrence`)  AS t2
                ON
                    t1.procedure_occurrence_id=t2.procedure_occurrence_id
            WHERE
                (t1.procedure_source_concept_id is null or t1.procedure_source_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
procedure_occurrence_procedure_source_concept_id.shape

print(procedure_occurrence_procedure_source_concept_id.shape[0], 'records received.')

Getting the data from the database...
32 records received.


In [125]:
procedure_occurrence_procedure_source_concept_id=procedure_occurrence_procedure_source_concept_id.rename(columns={"succes_rate":"procedure_occurrence_procedure_source_concept_id"})
procedure_occurrence_procedure_source_concept_id=procedure_occurrence_procedure_source_concept_id[["src_hpo_id","procedure_occurrence_procedure_source_concept_id"]]
procedure_occurrence_procedure_source_concept_id=procedure_occurrence_procedure_source_concept_id.fillna(100)
procedure_occurrence_procedure_source_concept_id

,src_hpo_id,procedure_occurrence_procedure_source_concept_id
0,jhchc,100.0
1,seec_ufl,100.0
2,syhc,100.0
3,saou_uab,100.0
4,ipmc_uic,100.0
5,trans_am_spectrum,100.0
6,nec_phs,100.0
7,cpmc_ceders,100.0
8,pitt,100.0
9,aouw_mcw,100.0


## Device Exposure

#### device_concept_id BY SITE

In [126]:
######################################
print('Getting the data from the database...')
######################################

device_exposure_device_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.device_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.device_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            WHERE
                (t1.device_concept_id is null or t1.device_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
device_exposure_device_concept_id.shape

print(device_exposure_device_concept_id.shape[0], 'records received.')

Getting the data from the database...
21 records received.


In [127]:
device_exposure_device_concept_id=device_exposure_device_concept_id.rename(columns={"succes_rate":"device_exposure_device_concept_id"})
device_exposure_device_concept_id=device_exposure_device_concept_id[["src_hpo_id","device_exposure_device_concept_id"]]
device_exposure_device_concept_id=device_exposure_device_concept_id.fillna(100)
device_exposure_device_concept_id

,src_hpo_id,device_exposure_device_concept_id
0,tach_hfhs,100.0
1,cpmc_uci,100.0
2,chs,100.0
3,aouw_mcw,100.0
4,nyc_cornell,100.0
5,ecchc,100.0
6,jhchc,100.0
7,syhc,100.0
8,cpmc_usc,100.0
9,pitt,100.0


#### device_type_concept_id BY SITE

In [128]:
######################################
print('Getting the data from the database...')
######################################

device_exposure_device_type_concept_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.device_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                `{}.concept` AS c
                ON
                    t1.device_type_concept_id=c.concept_id
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            WHERE
                (t1.device_type_concept_id is null or t1.device_type_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
device_exposure_device_type_concept_id.shape

print(device_exposure_device_type_concept_id.shape[0], 'records received.')

Getting the data from the database...
21 records received.


In [129]:
device_exposure_device_type_concept_id=device_exposure_device_type_concept_id.rename(columns={"succes_rate":"device_exposure_device_type_concept_id"})
device_exposure_device_type_concept_id=device_exposure_device_type_concept_id[["src_hpo_id","device_exposure_device_type_concept_id"]]
device_exposure_device_type_concept_id=device_exposure_device_type_concept_id.fillna(100)
device_exposure_device_type_concept_id

,src_hpo_id,device_exposure_device_type_concept_id
0,nyc_cu,100.0
1,cpmc_ceders,100.0
2,syhc,100.0
3,ipmc_uic,100.0
4,trans_am_spectrum,100.0
5,tach_hfhs,100.0
6,aouw_mcri,100.0
7,jhchc,100.0
8,saou_uab,100.0
9,aouw_uwh,100.0


#### provider_id BY SITE

In [130]:
######################################
print('Getting the data from the database...')
######################################

device_exposure_provider_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            WHERE
                (t1.provider_id is null or t1.provider_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
device_exposure_provider_id.shape

print(device_exposure_provider_id.shape[0], 'records received.')

Getting the data from the database...
21 records received.


In [131]:
device_exposure_provider_id=device_exposure_provider_id.rename(columns={"succes_rate":"device_exposure_provider_id"})
device_exposure_provider_id=device_exposure_provider_id[["src_hpo_id","device_exposure_provider_id"]]
device_exposure_provider_id=device_exposure_provider_id.fillna(100)
device_exposure_provider_id

,src_hpo_id,device_exposure_provider_id
0,hrhc,100.0
1,trans_am_spectrum,100.0
2,ipmc_uic,100.0
3,aouw_mcri,100.0
4,chs,100.0
5,cpmc_usc,0.0
6,nyc_cu,0.0
7,ecchc,0.0
8,cpmc_uci,0.0
9,saou_uab,0.0


#### visit_occurrence_id BY SITE

In [132]:
######################################
print('Getting the data from the database...')
######################################

device_exposure_visit_occurrence_id = pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            WHERE
                (t1.visit_occurrence_id is null or t1.visit_occurrence_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    ORDER BY
        4
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
device_exposure_visit_occurrence_id.shape

print(device_exposure_visit_occurrence_id.shape[0], 'records received.')

Getting the data from the database...
21 records received.


In [133]:
device_exposure_visit_occurrence_id=device_exposure_visit_occurrence_id.rename(columns={"succes_rate":"device_exposure_visit_occurrence_id"})
device_exposure_visit_occurrence_id=device_exposure_visit_occurrence_id[["src_hpo_id","device_exposure_visit_occurrence_id"]]
device_exposure_visit_occurrence_id=device_exposure_visit_occurrence_id.fillna(100)
device_exposure_visit_occurrence_id

,src_hpo_id,device_exposure_visit_occurrence_id
0,tach_hfhs,100.0
1,jhchc,100.0
2,seec_morehouse,100.0
3,aouw_mcw,100.0
4,syhc,100.0
5,hrhc,100.0
6,cpmc_usc,100.0
7,trans_am_spectrum,100.0
8,ipmc_uic,100.0
9,saou_uab,100.0


#### device_source_concept_id BY SITE

In [134]:
######################################
print('Getting the data from the database...')
######################################

device_exposure_device_source_concept_id= pd.io.gbq.read_gbq('''
    WITH
        hpo_counts AS (
            SELECT 
                src_hpo_id,
                COUNT(t1.person_id) as total_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            GROUP BY
                1
        ),

        hpo_missing_counts AS (
            SELECT
                t2.src_hpo_id,
                COUNT(t1.person_id) as missing_counts
            FROM
                `{}.unioned_ehr_device_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_device_exposure`)  AS t2
                ON
                    t1.device_exposure_id=t2.device_exposure_id
            WHERE
                (t1.device_source_concept_id is null or t1.device_source_concept_id=0) 
            GROUP BY
                1
        )

    SELECT
        hpo_counts.src_hpo_id,
        missing_counts,
        total_counts,
        round(100-100*(missing_counts/total_counts),1) AS succes_rate
    FROM
       hpo_counts
    FULL OUTER JOIN
        hpo_missing_counts
            ON
                hpo_missing_counts.src_hpo_id=hpo_counts.src_hpo_id
    '''.format(DATASET, DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                    dialect='standard')
device_exposure_device_source_concept_id.shape

print(device_exposure_device_source_concept_id.shape[0], 'records received.')

Getting the data from the database...
21 records received.


In [135]:
device_exposure_device_source_concept_id

,src_hpo_id,missing_counts,total_counts,succes_rate
0,cpmc_uci,NaN,19368,NaN
1,uamc_banner,171307.0,171307,0.0
2,jhchc,NaN,569,NaN
3,seec_morehouse,1153917.0,1153917,0.0
4,ipmc_uic,NaN,10371,NaN
5,trans_am_spectrum,271.0,271,0.0
6,tach_hfhs,NaN,20920,NaN
7,nyc_cu,4395.0,4395,0.0
8,aouw_mcw,68.0,68,0.0
9,saou_uab,59990.0,59990,0.0


In [136]:
device_exposure_device_source_concept_id=device_exposure_device_source_concept_id.rename(columns={"succes_rate":"device_exposure_device_source_concept_id"})
device_exposure_device_source_concept_id=device_exposure_device_source_concept_id[["src_hpo_id","device_exposure_device_source_concept_id"]]
device_exposure_device_source_concept_id=device_exposure_device_source_concept_id.fillna(100)
device_exposure_device_source_concept_id

,src_hpo_id,device_exposure_device_source_concept_id
0,cpmc_uci,100.0
1,uamc_banner,0.0
2,jhchc,100.0
3,seec_morehouse,0.0
4,ipmc_uic,100.0
5,trans_am_spectrum,0.0
6,tach_hfhs,100.0
7,nyc_cu,0.0
8,aouw_mcw,0.0
9,saou_uab,0.0


In [137]:
datas=[visit_occurrence_visit_type_concept_id,
visit_occurrence_provider_id_df,
visit_occurrence_care_site_id_df,
visit_occurrence_visit_source_concept_id_df,
visit_occurrence_admitting_source_concept_id_df,
visit_occurrence_discharge_to_concept_id_df,
visit_occurrence_preceding_visit_occurrence_id_df,
condition_occurrence_condition_concept_id_df,
condition_occurrence_condition_type_concept_id,
condition_occurrence_condition_source_concept_id,
condition_occurrence_condition_status_concept_id,
drug_exposure_drug_concept_id,
drug_exposure_drug_type_concept_id,
drug_exposure_route_concept_id,
drug_exposure_provider_id,
drug_exposure_visit_occurrence_id,
drug_exposure_drug_source_concept_id,
measurement_measurement_concept_id,
measurement_measurement_type_concept_id,
measurement_operator_concept_id,
measurement_value_as_concept_id,
measurement_unit_concept_id,
measurement_provider_id,
measurement_visit_occurrence_id,
measurement_measurement_source_concept_id,
procedure_occurrence_procedure_concept_id,
procedure_occurrence_procedure_type_concept_id,
procedure_occurrence_modifier_concept_id,
procedure_occurrence_provider_id,
procedure_occurrence_visit_occurrence_id,
procedure_occurrence_procedure_source_concept_id,
device_exposure_device_concept_id,
device_exposure_device_type_concept_id,
device_exposure_provider_id,
device_exposure_visit_occurrence_id,
device_exposure_device_source_concept_id]

In [138]:
master_df=visit_occurrence_visit_concept_id_df

In [139]:
for filename in datas:
    master_df = pd.merge(master_df,filename,on='src_hpo_id',how='outer')

In [140]:

master_df = pd.merge(master_df,site_df,on='src_hpo_id',how='outer')

In [141]:
master_df

,src_hpo_id,visit_occurrence_visit_concept_id,visit_occurrence_visit_type_concept_id,visit_occurrence_provider_id,visit_occurrence_care_site_id,visit_occurrence_visit_source_concept_id,visit_occurrence_admitting_source_concept_id,visit_occurrence_discharge_to_concept_id,visit_occurrence_preceding_visit_occurrence_id,condition_occurrence_condition_concept_id,condition_occurrence_condition_type_concept_id,condition_occurrence_condition_source_concept_id,condition_occurrence_condition_status_concept_id,drug_exposure_drug_concept_id,condition_occurrence_drug_type_concept_id,drug_exposure_route_concept_id,drug_exposure_provider_id,drug_exposure_visit_occurrence_id,drug_exposure_drug_source_concept_id,measurement_measurement_concept_id,measurement_measurement_type_concept_id,measurement_operator_concept_id,measurement_value_as_concept_id,measurement_unit_concept_id,measurement_provider_id,measurement_visit_occurrence_id,measurement_measurement_source_concept_id,procedure_occurrence_procedure_concept_id,procedure_occurrence_procedure_type_concept_id,procedure_occurrence_modifier_concept_id,procedure_occurrence_provider_id,procedure_occurrence_visit_occurrence_id,procedure_occurrence_procedure_source_concept_id,device_exposure_device_concept_id,device_exposure_device_type_concept_id,device_exposure_provider_id,device_exposure_visit_occurrence_id,device_exposure_device_source_concept_id,HPO
0,ecchc,100.0,100.0,0.0,0.0,0.0,0.0,NaN,0.0,100.0,100.0,100.0,NaN,100.0,100.0,NaN,0.0,75.4,100.0,88.5,100.0,81.3,0.0,55.3,0.0,0.2,88.5,100.0,100.0,0.0,0.0,100.0,100.0,100.0,100.0,0.0,0.0,100.0,Eau Claire Cooperative Health Center
1,ipmc_northshore,100.0,100.0,0.0,100.0,0.0,1.1,42.9,0.0,99.8,100.0,0.0,NaN,85.2,44.6,73.1,92.6,100.0,85.2,96.0,93.9,0.0,0.0,61.5,84.1,100.0,13.6,100.0,100.0,0.0,100.0,100.0,95.6,NaN,NaN,NaN,NaN,NaN,NorthShore University Health System
2,tach_hfhs,100.0,100.0,0.0,41.3,0.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,90.2,100.0,53.6,67.4,53.6,90.1,82.2,100.0,0.0,50.6,48.7,50.6,44.1,82.2,NaN,NaN,NaN,NaN,NaN,NaN,100.0,100.0,99.1,100.0,100.0,Henry Ford Health System
3,seec_morehouse,100.0,43.8,0.0,85.8,100.0,0.0,0.0,0.0,100.0,18.9,100.0,100.0,3.7,0.0,24.2,51.8,100.0,3.7,96.3,39.4,0.0,7.2,43.5,98.9,100.0,96.3,52.6,1.2,0.0,30.2,100.0,52.6,0.0,0.0,3.5,100.0,0.0,SouthEast Enrollment Center Morehouse
4,nyc_hh,100.0,51.8,0.0,0.0,0.0,100.0,NaN,1.4,100.0,100.0,100.0,100.0,100.0,99.5,98.8,0.0,100.0,100.0,100.0,100.0,89.8,NaN,NaN,0.0,95.2,100.0,100.0,100.0,0.0,0.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN,Harlem Hospital
5,hrhc,100.0,100.0,0.0,100.0,100.0,0.0,NaN,0.0,100.0,100.0,100.0,NaN,100.0,100.0,36.5,100.0,100.0,100.0,100.0,100.0,0.0,0.0,NaN,100.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,100.0,0.0,0.0,100.0,100.0,0.0,HRHCare
6,chci,100.0,100.0,0.0,100.0,0.0,0.0,NaN,0.0,100.0,100.0,100.0,NaN,100.0,100.0,99.7,0.0,100.0,100.0,100.0,100.0,0.0,0.0,100.0,0.0,100.0,100.0,100.0,85.9,0.0,0.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN,"Community Health Center, Inc."
7,syhc,100.0,100.0,0.0,96.6,0.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,100.0,100.0,0.0,99.3,100.0,100.0,100.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,91.1,100.0,100.0,100.0,100.0,83.2,100.0,0.0,San Ysidro Health Center
8,saou_uab,100.0,100.0,0.0,100.0,100.0,0.0,0.0,0.0,100.0,0.0,100.0,NaN,100.0,99.6,0.0,0.0,100.0,100.0,100.0,100.0,0.0,0.0,76.0,0.0,100.0,100.0,100.0,0.0,0.0,0.0,100.0,100.0,100.0,100.0,0.0,100.0,0.0,University of Alabama at Birmingham
9,ipmc_rush,100.0,100.0,0.0,100.0,0.0,100.0,99.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,0.0,0.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN,Rush University


In [142]:
master_df=master_df.fillna("No Data")
master_df

,src_hpo_id,visit_occurrence_visit_concept_id,visit_occurrence_visit_type_concept_id,visit_occurrence_provider_id,visit_occurrence_care_site_id,visit_occurrence_visit_source_concept_id,visit_occurrence_admitting_source_concept_id,visit_occurrence_discharge_to_concept_id,visit_occurrence_preceding_visit_occurrence_id,condition_occurrence_condition_concept_id,condition_occurrence_condition_type_concept_id,condition_occurrence_condition_source_concept_id,condition_occurrence_condition_status_concept_id,drug_exposure_drug_concept_id,condition_occurrence_drug_type_concept_id,drug_exposure_route_concept_id,drug_exposure_provider_id,drug_exposure_visit_occurrence_id,drug_exposure_drug_source_concept_id,measurement_measurement_concept_id,measurement_measurement_type_concept_id,measurement_operator_concept_id,measurement_value_as_concept_id,measurement_unit_concept_id,measurement_provider_id,measurement_visit_occurrence_id,measurement_measurement_source_concept_id,procedure_occurrence_procedure_concept_id,procedure_occurrence_procedure_type_concept_id,procedure_occurrence_modifier_concept_id,procedure_occurrence_provider_id,procedure_occurrence_visit_occurrence_id,procedure_occurrence_procedure_source_concept_id,device_exposure_device_concept_id,device_exposure_device_type_concept_id,device_exposure_provider_id,device_exposure_visit_occurrence_id,device_exposure_device_source_concept_id,HPO
0,ecchc,100,100,0,0,0,0,No Data,0,100,100,100,No Data,100,100,No Data,0,75.4,100,88.5,100,81.3,0,55.3,0,0.2,88.5,100,100,0,0,100,100,100,100,0,0,100,Eau Claire Cooperative Health Center
1,ipmc_northshore,100,100,0,100,0,1.1,42.9,0,99.8,100,0,No Data,85.2,44.6,73.1,92.6,100,85.2,96,93.9,0,0,61.5,84.1,100,13.6,100,100,0,100,100,95.6,No Data,No Data,No Data,No Data,No Data,NorthShore University Health System
2,tach_hfhs,100,100,0,41.3,0,0,0,0,100,100,100,0,90.2,100,53.6,67.4,53.6,90.1,82.2,100,0,50.6,48.7,50.6,44.1,82.2,No Data,No Data,No Data,No Data,No Data,No Data,100,100,99.1,100,100,Henry Ford Health System
3,seec_morehouse,100,43.8,0,85.8,100,0,0,0,100,18.9,100,100,3.7,0,24.2,51.8,100,3.7,96.3,39.4,0,7.2,43.5,98.9,100,96.3,52.6,1.2,0,30.2,100,52.6,0,0,3.5,100,0,SouthEast Enrollment Center Morehouse
4,nyc_hh,100,51.8,0,0,0,100,No Data,1.4,100,100,100,100,100,99.5,98.8,0,100,100,100,100,89.8,No Data,No Data,0,95.2,100,100,100,0,0,100,100,No Data,No Data,No Data,No Data,No Data,Harlem Hospital
5,hrhc,100,100,0,100,100,0,No Data,0,100,100,100,No Data,100,100,36.5,100,100,100,100,100,0,0,No Data,100,100,100,100,100,0,100,100,100,0,0,100,100,0,HRHCare
6,chci,100,100,0,100,0,0,No Data,0,100,100,100,No Data,100,100,99.7,0,100,100,100,100,0,0,100,0,100,100,100,85.9,0,0,100,100,No Data,No Data,No Data,No Data,No Data,"Community Health Center, Inc."
7,syhc,100,100,0,96.6,0,0,0,0,100,100,100,0,100,100,0,99.3,100,100,100,100,0,0,0,0,0,100,100,100,0,91.1,100,100,100,100,83.2,100,0,San Ysidro Health Center
8,saou_uab,100,100,0,100,100,0,0,0,100,0,100,No Data,100,99.6,0,0,100,100,100,100,0,0,76,0,100,100,100,0,0,0,100,100,100,100,0,100,0,University of Alabama at Birmingham
9,ipmc_rush,100,100,0,100,0,100,99,0,100,100,100,100,100,100,0,100,100,100,100,100,100,0,0,0,100,100,100,0,0,0,100,100,No Data,No Data,No Data,No Data,No Data,Rush University


In [143]:
master_df.to_csv("data\\foreign.csv")